# Calculating Bollinger Bands and Testing a Buy/Sell Strategy

* A `Bollinger Band` consists of a middle band (which is a moving average) and an upper and lower band. \
These upper and lower bands are set above and below the moving average by a certain number of standard deviations of price, thus incorporating volatility. \
The general principle is that by comparing a stock's position relative to the bands, a trader may be able to determine if a stock's price is relatively low or relatively high. \
Further, the width of the band can be an indicator of its volatility (narrower bands indicate less volatility while wider ones indicate higher volatility). (SCHWAB, 2023)

In [1]:
import backtrader as bt
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import yfinance as yf

# plt.style.use('seaborn')
# plt.style.use('seaborn-colorblind') #alternative
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Download Data
data = bt.feeds.PandasData(dataname=yf.download("AMZN", start="2020-01-01", end="2020-12-31"))

[*********************100%%**********************]  1 of 1 completed


1- The template of the strategy looks like

In [3]:
class BBand_Strategy(bt.Strategy):
    params = (('period', 20),
             ('devfactor', 2.0),)
    
    def __init__(self):
        # keep track of close price in the series
        self.data_close = self.datas[0].close
        self.data_open = self.datas[0].open
        
        # keep track of pending orders/buy price/buy commission
        self.order = None
        self.price = None
        self.comm = None
            
        # add Bollinger Bands indicator and track the buy/sell signals
        self.b_band = bt.ind.BollingerBands(self.datas[0], period=self.p.period, devfactor=self.p.devfactor)
        self.buy_signal = bt.ind.CrossOver(self.datas[0], self.b_band.lines.bot)
        self.sell_signal = bt.ind.CrossOver(self.datas[0], self.b_band.lines.top)
            
    def log(self, txt):
        '''Logging function'''''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return
        
        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Commission: {order.executed.comm:.2f}')
                self.price = order.executed.price
                self.comm = order.executed.comm
                
            else:
                self.log(f'SELL EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Commission: {order.executed.comm:.2f}')
                
        # report failed order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Failed')
        # set no pending order
            self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(f'OPERATION RESULT --- Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}')
        
    def next_open(self):
        if not self.position:
            if self.buy_signal > 0:
                # calculate the max number of shares ('all-in')
                size = int(self.broker.getcash()/self.datas[0].open)

                # buy order
                self.log(f'BUY CREATED --- Size: {size}, Cash: {self.broker.getcash():.2f}, Open: {self.data_open[0]:.2f}, Close: {self.data_close[0]:.2f}')
                self.buy(size=size)
                
        else:
            if self.sell_signal < 0:
                # sell order
                self.log(f'SELL CREATED --- Size: {self.position.size}')
                self.sell(size=self.position.size)

2- Set up the backtest

In [4]:
cerebro = bt.Cerebro(stdstats=False, cheat_on_open=True)

cerebro.adddata(data)
cerebro.addstrategy(BBand_Strategy)
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')

3- Run the backtest

In [5]:
print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')
backtest_result = cerebro.run()
print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

Starting Portfolio Value: 10000.00
2020-03-03, BUY CREATED --- Size: 101, Cash: 10000.00, Open: 98.77, Close: 95.45
2020-03-03, BUY EXECUTED --- Price: 98.77, Cost: 9975.62, Commission: 9.98
2020-04-20, SELL CREATED --- Size: 101
2020-04-20, SELL EXECUTED --- Price: 119.50, Cost: 9975.62, Commission: 12.07
2020-04-20, OPERATION RESULT --- Gross: 2093.63, Net: 2071.58
2020-11-03, BUY CREATED --- Size: 79, Cash: 12071.58, Open: 150.93, Close: 152.42
2020-11-03, BUY EXECUTED --- Price: 150.93, Cost: 11923.19, Commission: 11.92
2020-12-18, SELL CREATED --- Size: 79
2020-12-18, SELL EXECUTED --- Price: 162.20, Cost: 11923.19, Commission: 12.81
2020-12-18, OPERATION RESULT --- Gross: 890.57, Net: 865.83
Final Portfolio Value: 12937.41


4- Plot Result

In [6]:
cerebro.plot(iplot=False, volume=False)

[[<Figure size 2100x1019 with 4 Axes>]]

5- Run to investigate different returns metrics

In [7]:
print(backtest_result[0].analyzers.returns.get_analysis())

OrderedDict([('rtot', 0.25753828501126214), ('ravg', 0.0010219773214732624), ('rnorm', 0.2937413409339904), ('rnorm100', 29.37413409339904)])


6- Create a plot of daily portfolio returns

In [8]:
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
returns_dict = backtest_result[0].analyzers.time_return.get_analysis()
returns_df = pd.DataFrame(list(returns_dict.items()),
                          columns=['report_date', 'return']).set_index('report_date')
returns_df.plot(title='Portfolio Returns')

plt.tight_layout()
plt.show()